# Lab 4 - Cortex Agents Setup

This notebook creates Snowflake Cortex Agents for the services built in Labs 1-3 and sets up an email agent for Snowflake Intelligence summaries.

## What We'll Create
- **LAB1_SEARCH_AGENT**: Agent for Lab 1 Cortex Search service (document search)
- **LAB2_ANALYST_AGENT**: Agent for Lab 2 Cortex Analyst service (wealth management analytics)
- **LAB3_SEARCH_AGENT**: Agent for Lab 3 Cortex Search service (multimodal document parsing)
- **Email Infrastructure**: Stored procedure and scheduled task for Snowflake Intelligence summaries

## Prerequisites
- Completed Labs 1, 2, and 3 with their respective Cortex services
- Appropriate privileges to create agents, procedures, and tasks
- Email notification integration configured (for email functionality)

## References
- [Cortex Agents Documentation](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-agents)
- [Snowflake Intelligence Overview](https://docs.snowflake.com/en/user-guide/snowflake-cortex/snowflake-intelligence)
- [Snowflake Intelligence Tools](https://github.com/snowflake-procs-tg/snowflake-intelligence-tools/tree/main)


## 1. Configuration

First, let's set up our configuration and discover existing services from the previous labs.


In [ ]:
# Configuration - Update these values as needed
WAREHOUSE = "CORTEX_SEARCH_TUTORIAL_WH"
ROLE = "SYSADMIN"

# Lab 1 - Search Service Configuration
LAB1_DATABASE = "CORTEX_SEARCH_TUTORIAL_DB"
LAB1_SCHEMA = "PUBLIC"
LAB1_SEARCH_SERVICE = "DOCUMENT_SEARCH_SERVICE"

# Lab 2 - Analyst Service Configuration
LAB2_DATABASE = "CORTEX_ANALYST_DEMO"
LAB2_SCHEMA = "WEALTH_MANAGEMENT"
LAB2_ANALYST_SERVICE = "WEALTH_ANALYST_SERVICE"

# Lab 3 - Search Service Configuration
LAB3_DATABASE = "CORTEX_SEARCH_TUTORIAL_DB"
LAB3_SCHEMA = "PUBLIC"
LAB3_SEARCH_SERVICE = "DOCS_SEARCH_SERVICE"

# Email Configuration (optional)
EMAIL_ENABLED = True
EMAIL_INTEGRATION = "YOUR_EMAIL_INTEGRATION"  # Update with your integration name
EMAIL_RECIPIENT = "recipient@example.com"     # Update with recipient email
EMAIL_SCHEDULE = "USING CRON 0 9 * * * UTC"   # Daily at 9 AM UTC

print("Configuration loaded successfully!")
print(f"Warehouse: {WAREHOUSE}")
print(f"Role: {ROLE}")


## 2. Environment Setup

Set up the Snowflake environment and verify our services exist.


In [ ]:
-- Set up environment
USE ROLE IDENTIFIER($ROLE);
USE WAREHOUSE IDENTIFIER($WAREHOUSE);

SELECT 'Environment setup complete' as status;


## 3. Create Cortex Agents

Now let's create the Cortex Agents for each of our services. Agents provide a unified interface to interact with Cortex services through Snowflake Intelligence.

### 3.1 Lab 1 Search Agent

This agent will handle document search queries from Lab 1's Cortex Search service.


In [ ]:
-- Create Lab 1 Search Agent
CREATE OR REPLACE AGENT LAB1_SEARCH_AGENT
USING 'Cortex Search'
WITH PARAMETERS (
  'service_name' = $LAB1_SEARCH_SERVICE,
  'database' = $LAB1_DATABASE,
  'schema' = $LAB1_SCHEMA
);

SELECT 'Lab 1 Search Agent created successfully' as status;


### 3.2 Lab 2 Analyst Agent

This agent will handle wealth management analytics queries using the Cortex Analyst service from Lab 2.


In [ ]:
-- Create Lab 2 Analyst Agent
CREATE OR REPLACE AGENT LAB2_ANALYST_AGENT
USING 'Cortex Analyst'
WITH PARAMETERS (
  'service_name' = $LAB2_ANALYST_SERVICE,
  'database' = $LAB2_DATABASE,
  'schema' = $LAB2_SCHEMA
);

SELECT 'Lab 2 Analyst Agent created successfully' as status;


### 3.3 Lab 3 Search Agent

This agent will handle multimodal document parsing queries from Lab 3's enhanced Cortex Search service.


In [ ]:
-- Create Lab 3 Search Agent
CREATE OR REPLACE AGENT LAB3_SEARCH_AGENT
USING 'Cortex Search'
WITH PARAMETERS (
  'service_name' = $LAB3_SEARCH_SERVICE,
  'database' = $LAB3_DATABASE,
  'schema' = $LAB3_SCHEMA
);

SELECT 'Lab 3 Search Agent created successfully' as status;


## 4. Verify Agent Creation

Let's verify that all our agents were created successfully.


In [ ]:
-- Show all agents we just created
SHOW AGENTS LIKE '%LAB%_AGENT';


## 5. Email Infrastructure for Snowflake Intelligence

Now let's set up the email infrastructure to send periodic summaries of Snowflake Intelligence analyses.

### 5.1 Create Email Stored Procedure

This procedure will gather analysis summaries and send them via email.


In [ ]:
-- Create stored procedure for sending analysis emails
CREATE OR REPLACE PROCEDURE SEND_ANALYSIS_EMAIL(
    EMAIL_INTEGRATION STRING,
    RECIPIENT STRING
)
RETURNS STRING
LANGUAGE SQL
AS
$$
DECLARE
    analysis_summary STRING;
BEGIN
    -- Gather analysis summary from recent Cortex activity
    SELECT 
        'Snowflake Intelligence Summary for ' || CURRENT_DATE() || ':\n\n' ||
        'Total Cortex queries today: ' || COUNT(*) || '\n' ||
        'Unique users: ' || COUNT(DISTINCT USER_NAME) || '\n' ||
        'Most active database: ' || COALESCE(MODE(DATABASE_NAME), 'N/A') || '\n\n' ||
        'Lab Agents Activity:\n' ||
        '- LAB1_SEARCH_AGENT: Document search queries\n' ||
        '- LAB2_ANALYST_AGENT: Wealth management analytics\n' ||
        '- LAB3_SEARCH_AGENT: Multimodal document parsing\n\n' ||
        'Generated at: ' || CURRENT_TIMESTAMP()
    INTO analysis_summary
    FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
    WHERE START_TIME >= CURRENT_DATE()
    AND (QUERY_TEXT ILIKE '%CORTEX%' OR QUERY_TEXT ILIKE '%AGENT%');
    
    -- Send the email
    CALL SYSTEM$SEND_EMAIL(
        EMAIL_INTEGRATION,
        RECIPIENT,
        'Snowflake Intelligence Daily Summary - ' || CURRENT_DATE(),
        COALESCE(analysis_summary, 'No analysis data available for today.')
    );
    
    RETURN 'Email sent successfully at ' || CURRENT_TIMESTAMP();
EXCEPTION
    WHEN OTHER THEN
        RETURN 'Error sending email: ' || SQLERRM;
END;
$$;

SELECT 'Email procedure created successfully' as status;


### 5.2 Create Scheduled Task

This task will automatically run the email procedure on a schedule.


In [ ]:
-- Create scheduled task for email reports
-- Note: Update EMAIL_INTEGRATION and EMAIL_RECIPIENT in the config section before enabling
CREATE OR REPLACE TASK EMAIL_ANALYSIS_TASK
    WAREHOUSE = IDENTIFIER($WAREHOUSE)
    SCHEDULE = $EMAIL_SCHEDULE
AS
    CALL SEND_ANALYSIS_EMAIL($EMAIL_INTEGRATION, $EMAIL_RECIPIENT);

SELECT 'Email task created successfully (suspended by default)' as status;


### 5.3 Enable the Email Task (Optional)

Tasks are created in a suspended state by default. Run this cell when you're ready to activate automated emails.


In [ ]:
-- Enable the email task (uncomment when ready)
-- ALTER TASK EMAIL_ANALYSIS_TASK RESUME;

SELECT 'Email task enable cell ready (uncomment to activate)' as status;


## 6. Testing and Validation

Let's test our agents and email functionality.

### 6.1 Test Email Procedure (Optional)

**Note**: Only run this if you have configured a valid email integration.


In [ ]:
-- Test the email procedure (uncomment and update parameters if you want to test)
-- CALL SEND_ANALYSIS_EMAIL('YOUR_EMAIL_INTEGRATION', 'recipient@example.com');

SELECT 'Email test cell ready (uncomment to execute)' as status;


### 6.2 Monitor Task Status


In [ ]:
-- Check task status
SHOW TASKS LIKE 'EMAIL_ANALYSIS_TASK';


## 7. Usage in Snowflake Intelligence

🎉 **Congratulations!** Your agents are now ready to use in Snowflake Intelligence.

### How to Use Your Agents:

1. **Navigate to Snowflake Intelligence** in your Snowflake interface
2. **Select an agent** from the dropdown:
   - `LAB1_SEARCH_AGENT`: Document search from Lab 1
   - `LAB2_ANALYST_AGENT`: Wealth management analytics from Lab 2  
   - `LAB3_SEARCH_AGENT`: Multimodal document parsing from Lab 3

3. **Ask natural language questions** like:
   - For LAB1_SEARCH_AGENT: \"Find documents about pension considerations\"
   - For LAB2_ANALYST_AGENT: \"What's the total portfolio value by client segment?\"
   - For LAB3_SEARCH_AGENT: \"Analyze the 2023 factbook for investment trends\"

### Email Summaries:
- Configure your email integration and recipient in the configuration section
- Enable the task to receive daily summaries of Snowflake Intelligence activity
- Customize the email content by modifying the `SEND_ANALYSIS_EMAIL` procedure


## 8. Comprehensive Testing & Validation

Let's systematically test that everything works correctly before using the agents in production.

### 8.1 Prerequisites Check

First, verify that all the services from Labs 1-3 actually exist.
